# Time Series Regression Analysis

# Importing Necessary Packages

In [1]:
import os
import pyodbc
from dotenv import load_dotenv, dotenv_values, find_dotenv
import pandas as pd

## Loading Data From Database

In [2]:
#reading data from database
#Load environment variables from .env file into a dictionary variable
#.env file MUST be in the same root as the notebook to work or alternatively use find_dotenv
environment_variables=dotenv_values(find_dotenv('.env'))
# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")
 
#Connecting to the database
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
 
# Using the connect method of the pyodbc library.
# This will connect to the server.
connection=pyodbc.connect(connection_string)
 
#hide warnings
import warnings
 
warnings.filterwarnings('ignore')
 
print("connected successfully")

connected successfully


#### Adding Connected data to Database folder

In [3]:
#reading data from the remote database table
query='''SELECT *
        FROM dbo.oil'''
       
oil=pd.read_sql(query,connection)
 
oil.to_csv('../Datasets/oil.csv', index=False)

In [4]:
#reading data from the remote database table
query='''SELECT *
        FROM dbo.holidays_events'''
       
holidays=pd.read_sql(query,connection)
 
holidays.to_csv('../Datasets/holidays.csv', index=False)

In [5]:
#reading data from the remote database table
query='''SELECT *
        FROM dbo.stores'''
       
stores=pd.read_sql(query,connection)
 
stores.to_csv('../Datasets/stores.csv', index=False)

### Loading datasets to notebook as dataframes

In [13]:
# Adding all datasets as dataframes 
df_train = pd.read_csv("../Datasets/train.csv")
df_trans = pd.read_csv("../Datasets/transactions.csv")
df_holidays = pd.read_csv("../Datasets/holidays.csv")
df_stores = pd.read_csv("../Datasets/stores.csv")
df_oil = pd.read_csv("../Datasets/oil.csv")

# Data Understanding
Data understanding will follow the following order
1. Check for shape
2. Check for duplicates in data frames
3. Check for nulls in 
4. check data types in the columns
5. Run exploratory data analysis

#### Checking for Dataframe Shapes

In [7]:
data = {'train':df_train,'transactions':df_trans,'oil':df_oil,'stores':df_stores,'holidays':df_holidays}
shape = {}
 
for df in data:
    shape[df] = data[df].shape
   
print(pd.DataFrame(shape).T.rename(columns = {0:'rows',1:'columns'}))

                 rows  columns
train         3000888        6
transactions    83488        3
oil              1218        2
stores             54        5
holidays          350        6


### Checking for Nulls in dataframes

In [16]:
dataframes = {'train':df_train,'transactions':df_trans,'oil':df_oil,'stores':df_stores,'holidays':df_holidays}

def check_nulls(dataframes):
    for name, df in dataframes.items():
        null_count = df.isnull().sum().sum()
        if null_count == 0:
            print(f"No null values found in DataFrame {name}.")
        else:
            print(f"Null values found in DataFrame {name}: {null_count}.")
            
check_nulls(dataframes)


No null values found in DataFrame train.
No null values found in DataFrame transactions.
Null values found in DataFrame oil: 43.
No null values found in DataFrame stores.
No null values found in DataFrame holidays.


In [18]:
# Checking the oil  dataframe with null entries
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

### Checking for duplicates

In [34]:
def check_duplicates(dataframes):
    for name, df in dataframes.items():
        null_count = df.duplicated().sum().sum()
        if null_count == 0:
            print(f"No duplicate values found in DataFrame {name}.")
        else:
            print(f"duplicate values found in DataFrame {name}: {null_count}.")
            
check_duplicates(dataframes)

No duplicate values found in DataFrame train.
No duplicate values found in DataFrame transactions.
No duplicate values found in DataFrame oil.
No duplicate values found in DataFrame stores.
No duplicate values found in DataFrame holidays.


### Checking data frame infos

In [33]:
def check_dataframes_info(dataframes):
    for name, df in dataframes.items():
        print(f"DataFrame Info for {name}:")
        df.info()
        print("=" * 50)
check_dataframes_info(dataframes)

DataFrame Info for train:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB
DataFrame Info for transactions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
DataFrame Info for oil:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      -

In [8]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [9]:
df_trans.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [10]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [11]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [12]:
df_holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
